In [12]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [ ]:
#From the vacation search map, choose at least four cities in close 
#proximity on your map that are on the same continent 
#that a customer might travel to, and then create a directions layer map.

In [59]:
# Store the CSV you saved created in part one into a DataFrame.
preferred_cities_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
preferred_cities_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Rikitea,PF,75.65,overcast clouds,-23.12,-134.97,Pension Maro'i
1,1,Conakry,GN,87.80,haze,9.52,-13.70,Golden Plazza
2,2,Mindelo,PT,63.00,clear sky,41.31,-8.72,Santana Hotel & Spa
3,3,Comodoro Rivadavia,AR,71.60,few clouds,-45.87,-67.48,Austral Hotel
4,4,Navahrudak,BY,25.03,few clouds,53.59,25.82,Guest House in Novogrudok


In [66]:
City1 = preferred_cities_df[preferred_cities_df['City'].str.contains("Saint George")]
City1

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
30,33,Saint George,GR,42.8,broken clouds,39.45,22.34,MX PARK KYPARISSOS


In [67]:
City2 = preferred_cities_df[preferred_cities_df['City'].str.contains("Souillac")]
City2

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
171,197,Souillac,FR,46.4,overcast clouds,45.6,-0.6,Jolysable


In [71]:
City3 = preferred_cities_df[preferred_cities_df['City'].str.contains("Veraval")]
City3

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
217,247,Veraval,FR,39.2,broken clouds,49.65,0.71,Le Manoir aux Vaches


In [72]:
City4 = preferred_cities_df[preferred_cities_df['City'].str.contains("Ploemeur")]
City4

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
273,314,Ploemeur,FR,46.99,scattered clouds,47.73,-3.43,Best Western Plus Hotel Les Rives Du Ter


In [73]:
four_cities_df = pd.concat([City1, City2, City3, City4])
four_cities_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
30,33,Saint George,GR,42.80,broken clouds,39.45,22.34,MX PARK KYPARISSOS
171,197,Souillac,FR,46.40,overcast clouds,45.60,-0.60,Jolysable
217,247,Veraval,FR,39.20,broken clouds,49.65,0.71,Le Manoir aux Vaches
273,314,Ploemeur,FR,46.99,scattered clouds,47.73,-3.43,Best Western Plus Hotel Les Rives Du Ter


In [144]:
CityColumnsData = four_cities_df.loc[:,['City','Lat','Lng']]
CityColumnsData

,City,Lat,Lng
30,Saint George,39.45,22.34
171,Souillac,45.60,-0.60
217,Veraval,49.65,0.71
273,Ploemeur,47.73,-3.43


In [145]:
# import gmaps
import gmaps.datasets
from config import g_key

# Latitude-longitude pairs
SaintGeorge = (39.45,22.34)
Souillac = (45.60, -0.60)
Veraval = (49.65, 0.71)
Ploemeur = (47.73, -3.43)

fig = gmaps.figure()
tripPlanner = gmaps.directions_layer(SaintGeorge, Souillac, waypoints=[Veraval, Ploemeur])
fig.add_layer(tripPlanner)
fig

Figure(layout=FigureLayout(height='420px'))

In [147]:
fig = gmaps.figure()
tripPlanner_via_driving = gmaps.directions_layertripPlanner = gmaps.directions_layer(SaintGeorge, Souillac, waypoints=[Veraval, Ploemeur],travel_mode='DRIVING')
fig.add_layer(tripPlanner_via_driving)
fig

Figure(layout=FigureLayout(height='420px'))

In [148]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
import urllib3
urllib3.disable_warnings()
for index, row in four_cities_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params,verify=False).json()
    # Grab the first hotel from the results and store the name.
    try:
        four_cities_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [153]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
four_cities_hotel_info = [info_box_template.format(**row) for index, row in four_cities_df.iterrows()]

In [154]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = four_cities_df[["Lat", "Lng"]]
max_temp = four_cities_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=four_cities_hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))